教程地址 https://conda.io/projects/conda/en/latest/

# Concepts
+ Conda commands
+ Conda packages
+ Conda package specification
+ Conda channels
+ Conda environments
+ Installing with conda
+ Conda performance
+ Conda for data scientists

## Conda commands
作用：
+ conda命令可以查询并搜索Anaconda包索引和当前Anaconda安装
+ 创建新的conda环境
+ 在现有的conda环境中安装和更新包  

> Tips:  
> You can abbreviate many frequently used command options that are preceded by 2 dashes (--) to just 1 dash and the first letter of the option. So --name and -n are the same, and --envs and -e are the same.

command reference https://conda.io/projects/conda/en/latest/commands.html  
Viewing command-line help https://conda.io/projects/conda/en/latest/user-guide/tasks/view-command-line-help.html 

## Conda packages
### What is a conda package?
A conda package is a compressed tarball file (.tar.bz2) or .conda file that contains:
+ system-level libraries  
+ Python or other modules  
+ executable programs and other components  
+ metadata under the info/ directory  
+ a collection of files that are installed directly into an install prefix

### .conda file format
These compressed files can be significantly smaller than their bzip2 equivalents. In addition, they decompress much more quickly. .conda is the preferred file format to use where available, although we continue to provide .tar.bz2 files in tandem.
> Note  
> In conda 4.7 and later, you cannot use package names that end in “.conda” as they conflict with the .conda file format for packages.

### Using packages
+ You may search for packages
      conda search scipy
+ You may install a packages
      conda install scipy
+ You may build a package after installing conda build
      conda build my_fun_package
   

### Package structure
    .
    ├── bin
    │   └── pyflakes
    ├── info
    │   ├── LICENSE.txt
    │   ├── files
    │   ├── index.json
    │   ├── paths.json
    │   └── recipe
    └── lib
        └── python3.5

+ bin contains relevant binaries for the package
+ lib contains the relevant library files (eg. the .py files)
+ info contains package metadata

### Metapackages
当conda包仅用于元数据不含任何文件时，它被称为元包  
An example of a metapackage is "anaconda," which collects together all the packages in the Anaconda installer. The command conda create -n envname anaconda creates an environment that exactly matches what would be created from the Anaconda installer. You can create metapackages with the conda metapackage command. Include the name and version in the command.  
+ Anaconda metapackage  
The Anaconda metapackage is used in the creation of the Anaconda Distribution installers so that they have a set of packages associated with them.  
read more https://www.anaconda.com/whats-in-a-name-clarifying-the-anaconda-metapackage/
+ Mutex metapackage  
    A mutex metapackage is a very simple package that has a name.  
    Let's look at some examples for how to use mutex metapackages to build NumPy against different BLAS implementations.
    + Building NumPy with BLAS variants  
        If you build NumPy with MKL, you also need to build SciPy, scikit-learn, and anything else using BLAS also with MKL. It is important to ensure that these “variants” (packages built with a particular set of options) are installed together and never with an alternate BLAS implementation.
    + Installing NumPy with BLAS variants
        To specify which variant of NumPy that you want, you could potentially specify the BLAS library you want:
          conda install numpy mkl
        This pathway may lead to some ambiguity and solutions with mixed BLAS, so using the metapackage is recommended. To specify MKL-powered NumPy in a non-ambiguous way, you can specify the mutex package (either directly or indirectly):
          conda install numpy “blas=*=mkl”
    + Track_features  
        One of conda’s optimization goals is to minimize the number of track_features needed to specify the desired specs. By adding track_features to one or more of the options, conda will de-prioritize it or “weigh it down.” The lowest priority package is the one that would cause the most track_features to be activated in the environment. The default package among many variants is the one that would cause the least track_features to be activated.
    + More info  
        https://github.com/AnacondaRecipes/aggregate/blob/9635228/vs2017/meta.yaml#L24

### Noarch packages
Noarch packages are packages that are not architecture specific and therefore only have to be built once. Noarch packages are either generic or Python. Noarch generic packages allow users to distribute docs, datasets, and source code in conda packages  
#### Noarch Python
Noarch Python packages cut down on the overhead of building multiple different pure Python packages on different architectures and Python versions by sorting out platform and Python version-specific differences at install time.  
In order to qualify as a noarch Python package, all of the following criteria must be fulfilled:
+ No compiled extensions
+ No post-link or pre-link or pre-unlink scripts
+ No OS-specific build scripts
+ No python version specific requirements
+ No skips except for Python version. If the recipe is py3 only, remove skip statement and add version constraint on Python in host and run section.
+ 2to3 is not used
+ Scripts argument in setup.py is not used
+ If console_script entrypoints are in setup.py, they are listed in meta.yaml
+ No activate scripts
+ Not a dependency of conda

### Link and unlink scripts
You may optionally execute scripts before and after the link and unlink steps.  
### More information
Go deeper on how to manage packages. Learn more about package metadata, repository structure and index, and package match specifications at Package specifications.

## Conda package specification
### Package metadata
The info/ directory contains all metadata about a package. Files in this location are not installed under the install prefix. Although you are free to add any file to this directory, conda only inspects the content of the files discussed below.
+ Info
    + files  
      a list of all the files in the package (not included in info/)        
    + index.json  
      metadata about the package including platform, version, dependencies, and build info  
    + paths.json  
        a list of files in the package, along with their associated SHA-256, size in bytes, and the type of path (eg. hardlink vs. softlink)
+ info/index.json  
    This file contains basic information about the package, such as name, version, build string, and dependencies.
+ info/files  
    Lists all files that are part of the package itself, 1 per line. 
+ info/has_prefix  
    Optional file.Lists all files that contain a hard-coded build prefix or placeholder prefix, which needs to be replaced by the install prefix at installation time.
+ info/license.txt  
    Optional file. The software license for the package.
+ info/no_link  
    Optional file. Lists all files that cannot be linked - either soft-linked or hard-linked - into environments and are copied instead.
+ info/about.json  
    Optional file. Contains the entries in the about section of the meta.yaml file.
+ info/recipe  
    A directory containing the full contents of the build recipe.
+ meta.yaml.rendered  
    The fully rendered build recipe. See conda render.

### Repository structure and index
A conda repository - or channel - is a directory tree, usually served over HTTPS, which has platform subdirectories, each of which contain conda packages and a repository index. 
### Package match specifications
This match specification is not the same as the syntax used at the command line with conda install, such as conda install python=3.4. Internally, conda translates the command line syntax to the spec defined in this section.  
EXAMPLE: python=3.4 is translated to python 3.4\*.  
When using the command line, put double quotes around any package version specification that contains the space character or any of the following characters: <, >, \*, or /  
EXAMPLE:
>conda install numpy=1.11  
conda install numpy==1.11  
conda install "numpy>1.11"  
conda install "numpy=1.11.1|1.11.3"  
conda install "numpy>=1.8,<2"
### Version ordering
类VersionOrder（对象）实现版本字符串之间的顺序关系  
版本字符串可以包含常用的字母数字字符（A-Za-z0-9），通过点和下划线分隔成组件。不允许空白段（即两个连续的点、前导/尾随下划线）  
__支持的版本字符串__2  
Conda supports six types of version strings:
+ Release versions contain only integers, e.g. 1.0, 2.3.5.
+ Pre-release versions use additional letters such as a or rc, for example 1.0a1, 1.2.beta3, 2.3.5rc3.
+ Development versions are indicated by the string dev, for example 1.0dev42, 2.3.5.dev12.
+ Post-release versions are indicated by the string post, for example 1.0post1, 2.3.5.post2.
+ Tagged versions have a suffix that specifies a particular property of interest, e.g. 1.1.parallel. Tags can be added to any of the preceding 4 types. As far as sorting is concerned, tags are treated like strings in pre-release versions.
+ An optional local version string separated by + can be appended to the main (upstream) version string. It is only considered in comparisons when the main versions are equal, but otherwise handled in exactly the same manner.

__可预测的版本排序__  
为了获得可预测的版本顺序，关键是要保持给定包的版本号方案随时间的推移保持一致。Conda将预发布版本视为低于发布版本。

## Conda channels
### What is a "conda channel"?
Conda通道是存储包的位置。它们是托管和管理包的基础。Conda包是从远程通道下载的，这些通道是指向包含Conda包的目录的url。conda命令搜索一组默认频道，包将自动从https://repo.anaconda.com/pkgs/ 下载和更新。您可以修改自动搜索的远程频道。详见 https://conda.io/projects/conda/en/latest/user-guide/configuration/use-condarc.html#config-channels
### Specifying channels when installing packages
From the command line use --channel
> conda install scipy --channel conda-forge

You may specify multiple channels by passing the argument multiple times:
> conda install scipy --channel conda-forge --channel bioconda

优先级从左到右递减-第一个参数的优先级高于第二个参数  
From the command line use --override-channels 只搜索指定的通道，而不是在.condarc中配置的任何通道。这也会忽略conda的默认频道
> conda search scipy --channel file:/<path to\>/local-channel --override-channels
### Conda clone channel RSS feed
我们提供了一个RSS提要，它表示频道克隆所克隆的所有内容，现在可以在CDN（内容交付网络）后面使用。RSS提要显示在滚动的两周时间内发生的事情，对于查看包的位置或是否运行了同步非常有用。

## Conda environments
conda环境是包含已安装的conda包的特定集合的目录。例如，您可能有一个使用NumPy 1.7及其依赖项的环境，以及另一个使用numpy1.6进行遗留测试的环境。如果更改一个环境，则不会影响其他环境。您可以很容易地激活或停用环境，这就是您在它们之间切换的方式。您还可以通过向某人提供environment.yaml文件的副本来与他们共享您的环境。详见
https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html

# Tasks

## Managing environments
使用conda，您可以创建、导出、列出、删除和更新安装了不同版本Python和/或包的环境。在环境之间切换或移动称为激活环境。也可以共享环境文件。
### Creating an environment with commands
默认情况下，环境安装在conda目录的envs目录中。有关指定其他路径的信息，请参阅Specifying a location for an environment 或运行conda create--help。  
Use the terminal or an Anaconda Prompt for the following steps:
1. To create an environment:
> conda create --name myenv
>> Note  
Replace myenv with the environment name.
2. When conda asks you to proceed, type y:
> proceed ([y]/n)?

    这将在/envs/中创建myenv环境。  
    此环境使用的Python版本与您当前使用的Python版本相同，因为您没有指定版本。
3. To create an environment with a specific version of Python:
> conda create -n myenv python=3.6

4. To create an environment with a specific package:
> conda create -n myenv scipy  

    OR
> conda create -n myenv python  
conda install -n myenv scipy

5. To create an environment with a specific version of a package:
> conda create -n myenv scipy=0.15.0
    
    OR
> conda create -n myenv python  
conda install -n myenv scipy=0.15.0

6. To create an environment with a specific version of Python and multiple packages:
> conda create -n myenv python=3.6 scipy=0.15.0 astroid babel

要在每次创建新环境时自动安装pip或其他程序，请将默认程序添加到.condarc配置文件的“创建默认软件包”部分。每次创建新环境时都会安装默认包。如果不希望在特定环境中安装默认软件包，请使用--no default packages标志：
> conda create --no-default-packages -n myenv python

### Creating an environment from an environment.yml file
Use the terminal or an Anaconda Prompt for the following steps:
1. Create the environment from the environment.yml file:
> conda env create -f environment.yml

yml文件的第一行设置新环境的名称。有关详细信息，请参见Creating an environment file manually。
2. Activate the new environment: 
> conda activate myenv
3. Verify that the new environment was installed correctly:
> conda env list

    You can also use 
> conda info --envs

### Specifying a location for an environment
通过在创建环境时提供目标目录的路径，可以控制conda环境的位置。例如，以下命令将在当前工作目录的子目录envs中创建新环境：
> conda create --prefix ./envs jupyterlab=0.35 matplotlib=3.1 numpy=1.16

然后，使用用于激活按名称创建的环境的相同命令激活以前缀创建的环境：
> conda activate ./envs

在创建环境时指定项目目录的子目录的路径有以下好处：
+ 通过将环境包含为子目录，可以很容易地判断项目是否使用独立环境。
+ 它使您的项目更加独立，因为所有内容（包括所需的软件）都包含在单个项目目录中。
+ 可以对所有环境使用相同的名称。如果将所有环境保存在envs文件夹中，则必须为每个环境指定不同的名称。

将conda环境放置在默认envs文件夹之外时，需要注意一些事项:
+ Conda无法再使用--name标志找到您的环境。通常需要将--prefix标志与环境的完整路径一起传递才能找到环境。
+ 在创建conda环境时指定安装路径可以使命令提示符以活动环境的绝对路径而不是环境名称作为前缀。

使用前缀激活环境后，提示将类似于以下内容：
> (/Users/USER_NAME/research/data-science/PROJECT_NAME/envs) $

To remove this long prefix in your shell prompt, modify the env_prompt setting in your .condarc file:
> conda config --set env_prompt '({name})'

This will edit your .condarc file if you already have one or create a .condarc file if you do not.  
现在，命令提示符将显示活动环境的通用名称，即环境根文件夹的名称：
> cd project-directory  
> conda activate ./env  
(env) project-directory $

### Updating an environment
You may need to update your environment for a variety of reasons.  
For example, it may be the case that:
+ one of your core dependencies just released a new version (dependency version number update).
+ you need an additional package for data analysis (add a new dependency).
+ you have found a better package and no longer need the older package (add new dependency and remove old dependency).

如果发生这些情况，您只需相应地更新environment.yml文件的内容，然后运行以下命令
> conda env update --prefix ./env --file environment.yml  --prune

Note:
--prune选项使conda从环境中删除不再需要的任何依赖项。

### Cloning an environment
Use the terminal or an Anaconda Prompt for the following steps:  
1. You can make an exact copy of an environment by creating a clone of it:
>conda create --name myclone --clone myenv

    Note:用新环境的名称替换myclone。用要复制的现有环境的名称替换myenv。

2. To verify that the copy was made:
>conda info --envs

    In the environments list that displays, you should see both the source environment and the new copy.

### Building identical conda environments
可以使用显式规范文件在相同的操作系统平台上（在同一台计算机上或在不同的计算机上）构建相同的conda环境
Use the terminal or an Anaconda Prompt for the following steps:
1. Run conda list --explicit to produce a spec list such as:
> \# This file may be used to create an environment using:  
\#  conda create --name <env> --file <this file>  
\# platform: osx-64
\@EXPLICIT  
https://repo.continuum.io/pkgs/free/osx-64/mkl-11.3.3-0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/numpy-1.11.1-py35_0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/openssl-1.0.2h-1.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/pip-8.1.2-py35_0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/python-3.5.2-0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/readline-6.2-2.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/setuptools-25.1.6-py35_0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/sqlite-3.13.0-0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/tk-8.5.18-0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/wheel-0.29.0-py35_0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/xz-5.2.2-0.tar.bz2
https://repo.continuum.io/pkgs/free/osx-64/zlib-1.2.8-3.tar.bz2
2. 要将此等级库列表创建为当前工作目录中的文件，请运行：
> conda list --explicit > spec-file.txt  

    Note:You can use spec-file.txt as the filename or replace it with a filename of your choice.

显式spec文件通常不是跨平台的，因此在顶部有一个注释，比如#platform:osx-64，显示了创建它的平台。这个平台就是这个spec文件的工作平台。在其他平台上，指定的包可能不可用，或者规范中已存在的某些关键包可能缺少依赖项  
To use the spec file to create an identical environment on the same machine or another machine:  
> conda create --name myenv --file spec-file.txt

To use the spec file to install its listed packages into an existing environment:
> conda install --name myenv --file spec-file.txt

从规范文件安装时，Conda不检查体系结构或依赖项。要确保包正常工作，请确保文件是从工作环境中创建的，并在相同的体系结构、操作系统和平台（如linux-64或osx-64）上使用它。
### Activating an environment
激活环境对于使环境中的软件正常工作至关重要。激活包含两个主要功能：
+ 向环境的路径添加条目
+ 以及运行环境可能包含的任何激活脚本

您还可以使用配置API设置环境变量  
安装Anaconda时，可以选择“将Anaconda添加到我的路径环境变量中”。不建议这样做，因为“添加到路径”选项会将Anaconda附加到路径。当安装程序附加到PATH时，它不会调用激活脚本  
在Windows下，路径由系统路径和用户路径两部分组成。系统路径总是第一位的。当你只为我安装Anaconda时，我们会将其添加到用户路径。为所有用户安装时，我们会将其添加到系统路径。在前一种情况下，系统路径值可能会优先于我们的条目。在后一种情况下，你不会。我们不建议多用户安装  
激活在路径前面。这仅在环境处于活动状态时生效，因此它是终端会话的本地会话，而不是全局会话。  
To activate an environment: 
> conda activate myenv

Conda将路径名myenv放在系统命令前面  
如果尚未激活环境，则可能会收到警告消息：
> Warning:
This Python interpreter is in a conda environment, but the environment has
not been activated. Libraries may fail to load. To activate this environment
please see https://conda.io/activation.

If you receive this warning, you need to activate your environment.  
To do so on Windows, run: c:\Anaconda3\Scripts\activate base in Anaconda Prompt.  
Windows对正确的激活非常敏感。这是因为Windows库加载器不支持库和可执行文件的概念，这些库和可执行文件知道在哪里搜索它们的依赖项（RPATH）。相反，Windows依赖于动态链接库搜索顺序。  
如果环境不处于活动状态，则找不到库，并且会出现许多错误。当子环境中的Python找不到必需的OpenSSL库时，HTTP或SSL错误是常见的错误。  
Conda本身包含一些特殊的解决方法来添加必要的路径条目。这使得它可以在不激活或任何子环境处于活动状态时调用。通常，在未首先激活环境的情况下调用环境中的任何可执行文件都可能无法工作。为了能够在激活的环境中运行可执行文件，您可能对conda run命令感兴趣。  
如果您遇到路径错误，请查看我们的故障排除https://conda.io/projects/conda/en/latest/user-guide/troubleshooting.html#path-error
#### Conda init
conda的早期版本引入了脚本，使激活行为在操作系统中统一。Conda 4.4允许conda activate myenv。Conda 4.6增加了大量的初始化支持，使得Conda在各种各样的shell（bash、zsh、csh、fish、xonsh等等）上运行得更快，中断更少。现在这些shell可以使用conda activate命令。不需要修改路径会减少conda对系统上其他软件的干扰。有关详细信息，请阅读conda init--help的输出  
One setting may be useful to you when using conda init is:  
>auto_activate_base: bool

此设置控制conda首次启动时是否激活基本环境。无论哪种方式，都可以使用conda命令，但如果不激活环境，环境中的其他程序都将不可用，直到使用conda activate base激活环境。人们有时选择此设置是为了加快启动shell所需的时间，或者是为了防止conda安装的软件自动隐藏其他软件  
#### Nested activation
默认情况下，conda activate将在激活新环境之前停用当前环境，并在停用新环境时重新激活它。有时您可能希望保留当前环境路径条目，以便可以继续从第一个环境轻松访问命令行程序。在基本环境中安装通用命令行实用程序时，最常见的情况是这样。要在路径中保留当前环境，可以使用以下方法激活新环境：
> conda activate --stack myenv

如果希望在从最外层环境（通常是基本环境）进入时始终堆栈，可以设置atuo_stack配置选项：
> conda config --set auto_stack 1

您可以为更深层次的自动堆叠指定更大的数字，但不建议这样做，因为更深层次的堆叠更容易导致混淆  
#### 用于DLL加载验证的环境变量
如果您不想激活您的环境并且希望Python为DLL加载验证工作，那么请遵循故障排除https://conda.io/projects/conda/en/latest/user-guide/troubleshooting.html#path-error

### 停用环境
To deactivate an environment, type: 
> conda deactivate

Conda从系统命令中删除当前活动环境的路径名  
Note:要简单地返回到基本环境，最好在没有指定环境的情况下调用conda activate，而不是尝试去激活。如果从基本环境运行conda deactivate，则可能会完全失去运行conda的能力。别担心，这是这个shell的本地特性-你可以开始一个新的shell。但是，如果环境是使用--stack激活的（或者是自动堆叠的），那么最好使用conda deactivate。

### Determining your current environment
Use the terminal or an Anaconda Prompt for the following steps  
默认情况下，当前使用的活动环境显示在命令提示符开头的括号（）或括号[]中：
> (myenv) ￥

If you do not see this, run:
> conda info --envs

在显示的“环境”列表中，当前环境将以星号（\*）亮显  
默认情况下，命令提示设置为显示活动环境的名称。要禁用此选项：  
> conda config --set changeps1 false

To re-enable this option:  
> conda config --set changeps1 true

### Viewing a list of your environments
To see a list of all of your environments, in your terminal window or an Anaconda Prompt, run:
> conda info --envs

OR
> conda env list

将显示类似于以下内容的列表：
>conda environments:  
myenv                 /home/username/miniconda/envs/myenv  
snowflakes            /home/username/miniconda/envs/snowflakes  
bunnies               /home/username/miniconda/envs/bunnies

If this command is run by an administrator, a list of all environments belonging to all users will be displayed.  
### Viewing a list of the packages in an environment
To see a list of all packages installed in a specific environment:  
+ If the environment is not activated, in your terminal window or an Anaconda Prompt, run:
    > conda list -n myenv
+ If the environment is activated, in your terminal window or an Anaconda Prompt, run:
    > conda list

要查看特定软件包是否安装在环境中，请在终端窗口或Anaconda提示符下运行：
> conda list -n myenv scipy

### Using pip in an environment
To use pip in your environment, in your terminal window or an Anaconda Prompt, run:
>conda install -n myenv pip  
conda activate myenv  
pip pip_subcommand  

在一起使用pip和conda时可能会出现问题。在组合conda和pip时，最好使用隔离的conda环境。只有在使用conda安装尽可能多的软件包之后，才能使用pip安装任何剩余的软件。如果需要修改环境，最好创建一个新环境，而不是在pip之后运行conda。适当时，conda和pip需求应存储在文本文件中  
We recommend that you:
+ Use pip only after conda
    + Install as many requirements as possible with conda then use pip
    + Pip should be run with --upgrade-strategy only-if-needed (the default)
    + Do not use pip with the --user argument, avoid all “users” installs
+ Use conda environments for isolation
    + 创建conda环境以隔离pip所做的任何更改
    + 由于硬链接，环境占用的空间很小
    + 应注意避免在“root”环境中运行pip
+ 如果需要更改，请重新创建环境
    + 一旦使用pip，conda将不知道这些变化
    + To install additional conda packages, it is best to recreate the environment
+ 在文本文件中存储conda和pip需求
    + Package requirements can be passed to conda via the --file argument
    + Pip accepts a list of Python packages with -r or --requirements
    + Conda env will export or create environments based on a file with conda and pip requirements

### Setting environment variables
如果要将环境变量与环境关联，可以使用配置API。这是建议使用激活和停用脚本的替代方法，因为这些脚本是执行可能不安全的任意代码  
First, create your environment and activate it:
> conda create -n test-env  
conda activate test-env

To list any variables you may have, run 
> conda env config vars list

To set environment variables, run 
> conda env config vars set my_var=value

设置环境变量后，必须重新激活环境
> conda activate test-env

To check if the environment variable has been set, run 
> echo my_var 

or 
> conda env config vars list

停用环境时，可以使用这些相同的命令来查看环境变量是否消失  
可以使用-n和-p标志指定要影响的环境。n标志允许您命名环境，-p允许您指定环境的路径  
To unset the environment variable, run:
> conda env config vars unset my_var -n test-env

当您停用环境时，可以通过重新运行echo my_var或conda env config vars list来显示变量名不再存在，从而看到该环境变量消失
### Saving environment variables
Conda环境可以包含保存的环境变量  
这种类型的脚本文件可以是conda包的一部分，在这种情况下，当包含该包的环境被激活时，这些环境变量将变为活动的  
你可以随意命名这些脚本,一个流行的选项是以packagename-script name.sh的形式为脚本命名，或者在Windows上为packagename-scriptname.bat
+ Windows
    1. 在命令shell%conda_PREFIX%中运行，在Anaconda提示符中找到conda环境的目录
    2. 输入该目录并创建这些子目录和文件
        > cd %CONDA_PREFIX%  
        mkdir .\etc\conda\activate.d  
        mkdir .\etc\conda\deactivate.d  
        type NUL > .\etc\conda\activate.d\env_vars.bat  
        type NUL > .\etc\conda\deactivate.d\env_vars.bat  
    3. Edit .\etc\conda\activate.d\env_vars.bat as follows:
        > set MY_KEY='secret-key-value'  
        set MY_FILE=C:\path\to\my\file
    4. Edit .\etc\conda\deactivate.d\env_vars.bat as follows:
        > set MY_KEY=  
        set MY_FILE=
        
  When you run conda activate analytics, the environment variables MY_KEY and MY_FILE are set to the values you wrote into the file. When you run conda deactivate, those variables are erased.
+ macOS and Linux
    1. Locate the directory for the conda environment in your terminal window by running in the terminal echo ￥CONDA_PREFIX.
    2. Enter that directory and create these subdirectories and files:
        > cd ￥CONDA_PREFIX  
        mkdir -p ./etc/conda/activate.d  
        mkdir -p ./etc/conda/deactivate.d  
        touch ./etc/conda/activate.d/env_vars.sh  
        touch ./etc/conda/deactivate.d/env_vars.sh
    3. Edit ./etc/conda/activate.d/env_vars.sh as follows:
        >#!/bin/sh  
        export MY_KEY='secret-key-value'  
        export MY_FILE=/path/to/my/file/  
    4. Edit ./etc/conda/deactivate.d/env_vars.sh as follows:
        >#!/bin/sh  
        unset MY_KEY
        unset MY_FILE

  运行conda activate analytics时，环境变量MY_KEY和MY_FILE将设置为您写入文件的值。当运行conda deactivate时，这些变量将被删除
  
### Sharing an environment
您可能希望与其他人共享您的环境，例如，这样他们可以重新创建您已经完成的测试。为了让他们能够快速复制您的环境及其所有包和版本，请给他们一个environment.yml文件的副本
#### Exporting the environment.yml file
Note:If you already have an environment.yml file in your current directory, it will be overwritten during this task.
1. Activate the environment to export: conda activate myenv
2. Export your active environment to a new file
    > conda env export > environment.yml
    
    Note:This file handles both the environment's pip packages and conda packages.
3. Email or copy the exported environment.yml file to the other person

#### 跨平台导出环境文件
If you want to make your environment file work across platforms, you can use the 
>conda env export --from-history 

flag. This will only include packages that you’ve explicitly asked for, as opposed to including every package in your environment.  
For example, if you create an environment and install Python and a package:
> conda install python=3.7 codecov

这将下载并安装许多额外的软件包来解决依赖关系。这将引入跨平台不兼容的包  
If you use conda env export, it will export all of those packages. However, if you use conda env export --from-history, it will only export those you specifically chose:
> (env-name) ➜  ~ conda env export --from-history  
> name: env-name  
> channels:  
>  \- conda-forge  
>  \- defaults  
> dependencies:  
>  \- python=3.7  
>  \- codecov  
> prefix: /Users/username/anaconda3/envs/env-name

#### 手动创建环境文件
You can create an environment file (environment.yml) manually to share with others  
EXAMPLE: A simple environment file:
> name: stats  
> dependencies:  
>  \- numpy  
>  \- pandas  

通过将nodefaults添加到“通道”列表，可以排除默认通道
> channels:  
>  \- javascript  
>  \- nodefaults

这相当于将--override channels选项传递给大多数conda命令  
有关从此environment.yml文件创建环境的详细信息，请参见从environment.yml文件创建环境

### Restoring an environment
Conda保留了对环境所做的所有更改的历史记录，因此您可以轻松地“回滚”到以前的版本。列出当前环境中每个更改的历史记录：
> conda list --revisions

要将环境还原到以前的版本，请执行以下操作：
> conda install --revision=REVNUM

OR
> conda install --rev REVNUM

Note:Replace REVNUM with the revision number.  
Example: If you want to restore your environment to revision 8, run 
> conda install --rev 8

### Removing an environment
要删除环境，请在终端窗口或Anaconda提示符下运行：
> conda remove --name myenv --all

You may instead use 
> conda env remove --name myenv

To verify that the environment was removed, in your terminal window or an Anaconda Prompt, run:
> conda info --envs

The environments list that displays should not show the removed environment.

## Managing packages
+ Searching for packages
+ Installing packages
+ Installing similar packages
+ Installing packages from Anaconda.org
+ Installing non-conda packages
+ Installing commercial packages
+ Viewing a list of installed packages
+ Listing package dependencies
+ Updating packages
+ Preventing packages from updating (pinning)
+ Adding default packages to new environments automatically
+ Removing packages

### Searching for packages
Use the terminal or an Anaconda Prompt for the following steps:  
要查看是否有特定的软件包（如SciPy）可用于安装，请执行以下操作：  
> conda search scipy

To see if a specific package, such as SciPy, is available for installation from Anaconda.org:
> conda search --override-channels --channel defaults scipy

To see if a specific package, such as iminuit, exists in a specific channel, such as http://conda.anaconda.org/mutirri, and is available for installation:
> conda search --override-channels --channel http://conda.anaconda.org/mutirri iminuit  

### Installing packages
Use the terminal or an Anaconda Prompt for the following steps  
To install a specific package such as SciPy into an existing environment "myenv":
> conda install --name myenv scipy

If you do not specify the environment name, which in this example is done by --name myenv, the package installs into the current environment:
> conda install scipy

To install a specific version of a package such as SciPy:
> conda install scipy=0.15.0

To install multiple packages at once, such as SciPy and cURL:
> conda install scipy curl

Note:It is best to install all packages at once, so that all of the dependencies are installed at the same time  
To install multiple packages at once and specify the version of the package:
> conda install scipy=0.15.0 curl=7.26.0

To install a package for a specific Python version:
> conda install scipy=0.15.0 curl=7.26.0 -n py34_env

如果要使用特定的Python版本，最好将环境与该版本一起使用。有关详细信息，请参阅疑难解答

### Installing similar packages
安装具有类似文件名并具有类似用途的包可能会返回意外结果。最后安装的软件包可能会决定结果，这可能是不可取的。如果这两个包有不同的名称，或者您正在构建包的变体，并且需要在堆栈中排列其他软件，我们建议使用互斥元包
### Installing packages from Anaconda.org
使用conda安装无法获得的包可以从Anaconda.org获得，Anaconda.org是一个用于公共和私有包存储库的包管理服务。org是一种Anaconda 产品，就像Anaconda 和Miniconda一样  
To install a package from Anaconda.org:
1. In a browser, go to https://anaconda.org  
2. 要查找名为“bottleneck”的包，请在名为“Search Packages”的左上角框中键入“bottleneck”
3. Find the package that you want and click it to go to the detail page  
    “详细信息”页显示频道的名称。在本例中，它是“pandas”频道
4. 现在您知道了通道名称，请使用conda install命令安装该软件包。在终端窗口或蟒蛇提示中，运行:
    > conda install -c pandas bottleneck
    
   This command tells conda to install the bottleneck package from the pandas channel on Anaconda.org.
5. To check that the package is installed, in your terminal window or an Anaconda Prompt, run:
    > conda list
    
   A list of packages appears, including bottleneck.

### Installing non-conda packages
如果conda或Anaconda.org上没有软件包，您可以通过conda-forge或其他软件包管理器（如pip）找到并安装该软件包  
Pip软件包不具备conda软件包的所有功能，我们建议首先尝试使用conda安装任何软件包。如果通过conda无法获得包，请尝试使用conda-forge查找并安装它  
如果仍然无法安装该软件包，可以尝试使用pip安装它。pip和conda包之间的差异导致了某些不可避免的兼容性限制，但是conda很难尽可能地与pip兼容  
Note Conda environments replace virtualenv, so there is no need to activate a virtualenv before using pip  
可以在conda环境外部或conda环境内部安装pip5  
要获得conda集成的好处，请确保在当前活动的conda环境中安装pip，然后使用pip实例安装软件包。命令conda list显示以这种方式安装的包，标签显示它们是用pip安装的  
You can install pip in the current conda environment with the command conda install pip, as discussed in Using pip in an environment.  
如果当前conda环境内外都安装了pip实例，则使用当前conda环境内安装的pip实例  
To install a non-conda package:  
1. Activate the environment where you want to put the program:
    + On Windows, in your Anaconda Prompt, run activate myenv
    + On macOS and Linux, in your terminal window, run conda activate myenv
    
    5  
2. To use pip to install a program such as See, in your terminal window or an Anaconda Prompt, run:
    > pip install see

3. To verify the package was installed, in your terminal window or an Anaconda Prompt, run:
    > conda list
    
   If the package is not shown, install pip as described in Using pip in an environment and try these commands again.
   
### 安装商业软件包
Installing a commercial package such as IOPro is the same as installing any other package. In your terminal window or an Anaconda Prompt, run:
> conda install --name myenv iopro

这个命令安装了一个名为IOPro的商业软件包的免费试用版，可以加速Python的处理。除学术用途外，此免费试用期在30天后到期5
### Viewing a list of installed packages
Use the terminal or an Anaconda Prompt for the following steps.  
To list all of the packages in the active environment:
> conda list

To list all of the packages in a deactivated environment:
> conda list -n myenv

### Listing package dependencies
要查找哪些包取决于您环境中的特定包，没有一个特定的conda命令。它需要一系列步骤:
1. List the dependencies that a specific package requires to run: 
> conda info package_name

2. Find your installation’s package cache directory: conda info  
3. 查找包依赖项。默认情况下，Anaconda/Miniconda将软件包存储在macOS Catalina上的~/Anaconda/pkgs/（或~/opt/pkgs/）中。每个包都有一个index.json文件，其中列出了包的依赖项。此文件位于~anaconda/pkgs/package_name/info/index.json中
4. Now you can find what packages depend on a specific package. Use grep to search all index.json files as follows: 
> grep package_name ~/anaconda/pkgs/\*/info/index.json

The result will be the full package path and version of anything containing the 《package_name》.  